In [3]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding3D, BatchNormalization, Flatten, Conv3D, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
%matplotlib inline

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:435: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [20]:
def identity_block(X, f, filters, stage, block):


    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv3D(filters=F1, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = CSN_3D_BLOCK(X,F1)



    # Second component of main path (≈3 lines)
    X = Conv3D(filters=F2, kernel_size=(1,f, f), strides=(1,1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = CSN_3D_BLOCK(X,F2)


    # Third component of main path (≈2 lines)
    X = Conv3D(filters=F3, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2c')(X)
    
    X = CSN_3D_BLOCK(X,F3)


    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)


    return X

In [21]:
def CSN_3D_BLOCK(X,F1=None):

  # print(X.shape ,"csn 1")
  o1,o2,o3,o4,o5  = X.shape[2],X.shape[3],X.shape[4]*X.shape[1],X.shape[1],X.shape[4]
  # print(o1,o2,o3,o4,o5 ,"csn 2")

  X = Reshape((o1,o2,o3))(X)
  X = SeparableConv2D(F1,3,strides=1,padding='same')(X)
  # print(X.shape,"hendri check")
  X = Reshape((o4,o1,o2,X.shape[3]//o4))(X)
  
  return X

In [22]:
def convolutional_block(X, f, filters, stage, block, s=2):


    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv3D(filters=F1, kernel_size=(1,1, 1), strides=(1,s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = CSN_3D_BLOCK(X,F1)


    # Second component of main path (≈3 lines)
    X = Conv3D(filters=F2, kernel_size=(1,f, f), strides=(1,1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    X = CSN_3D_BLOCK(X,F2)


    # Third component of main path (≈2 lines)
    X = Conv3D(filters=F3, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name=bn_name_base + '2c')(X)

    X = CSN_3D_BLOCK(X,F3)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv3D(filters=X.shape[4], kernel_size=(1,1, 1), strides=(1,s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization( name=bn_name_base + '1')(X_shortcut)
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

 
    return X


In [23]:
def ResNet50(input_shape=(10,32, 32,3), classes=6):


    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    X =X_input

    # Stage 1
    X = Conv3D(64, (1,5, 5), strides=(1,2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization( name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling3D((1,3, 3), strides=(1,2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[32, 32, 128], stage=2, block='a', s=1)
    X = identity_block(X, 3, [32, 32, 128], stage=2, block='b')
    X = identity_block(X, 3, [32, 32, 128], stage=2, block='c')



    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=3, block='a', s=2)
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='c')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=4, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='d')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='e')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=5, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=5, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling3D(pool_size=(1,2, 2), padding='same')(X)

    # X = GlobalAveragePooling3D()(X)

 

    # output layer
    X = Flatten()(X)
    X = Dense(classes, name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [24]:
model = ResNet50(input_shape=(8,172, 172, 3), classes=101)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 8, 172, 172, 0                                            
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 8, 84, 84, 64 4864        input_5[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 8, 84, 84, 64 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_196 (Activation)     (None, 8, 84, 84, 64 0           bn_conv1[0][0]                   
___________________________________________________________________________________________